In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

2025-04-19 21:24:12.927395: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-19 21:24:12.984866: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-19 21:24:12.984918: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-19 21:24:12.986835: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-19 21:24:12.996226: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-19 21:24:12.996875: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [5]:
# Load the model
model = load_model('model.h5')

# Load the one hot encoded data and scaler
with open("label_encoder_gender.pkl", 'rb') as f:
    label_encoder_gender = pickle.load(f)

with open("onehot_encoder_geography.pkl", 'rb') as f:
    onehot_encoder_geography = pickle.load(f)

with open("scaler.pkl", 'rb') as f:
    scaler = pickle.load(f)




In [12]:
# Not likely to churn.

# input_data = {
#     'CreditScore': 619,
#     'Geography': 'France',
#     'Gender': 'Female',
#     'Age': 42,
#     'Tenure': 2,
#     'Balance': 0,
#     'NumOfProducts': 1,
#     'HasCrCard': 1,
#     'IsActiveMember': 1,
#     'EstimatedSalary': 101348.88
# }

# Likely to churn.
input_data = {   
   "CreditScore": 376,
   "Geography": "Germany",
   "Gender": "Female",  
   "Age": 29,
   "Tenure": 4,     
   "Balance": 115046.74,
   "NumOfProducts": 4,
   "HasCrCard": 1,
   "IsActiveMember": 0,
   "EstimatedSalary": 119346.88
}




In [13]:
# OneHot encode the geography column
geography_encoded = onehot_encoder_geography.transform([[input_data['Geography']]]).toarray()

# Convert the encoded array to a dataframe
geography_encoded_df = pd.DataFrame(geography_encoded,columns=onehot_encoder_geography.get_feature_names_out(['Geography']))
geography_encoded_df

/home/murli/murli/git_projects/employee_churn_modeling/myenv/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,1.0,0.0


In [14]:
# Convert the input data to a dataframe

input_df = pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,376,Germany,Female,29,4,115046.74,4,1,0,119346.88


In [ ]:
# Label encode the gender column
input_df['Gender'] = label_encoder_gender.transform([input_data['Gender']])

# Concatenate the input dataframe with the encoded geography dataframe
input_df = pd.concat([input_df.drop('Geography',axis=1), geography_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,376,0,29,4,115046.74,4,1,0,119346.88,0.0,1.0,0.0


In [ ]:
# Scale the input data
input_scaled = scaler.transform(input_df)

In [17]:
# Make a prediction

prediction = model.predict(input_scaled)
prediction


1/1 [==============================] - 0s 86ms/step


array([[0.9996688]], dtype=float32)

In [18]:
prediction_probabaility = prediction[0][0]
prediction_probabaility

0.9996688

In [19]:
if prediction_probabaility > 0.5:
    print("The customer is likely to churn.")
else:
    print("The customer is not likely to churn.")

The customer is likely to churn
